In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/Colab Notebooks/Thesis Project/

!pip install transformers
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Mounted at /content/drive/
[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/Thesis Project/'
/content


In [ ]:
print(torch.__version__)

2.5.0+cu121


### Define Hyperparameter

In [ ]:
# Define batch size
BATCH_SIZE = 32
LEARNING_RATE = 0.01

DROPOUT = 0.1

# Training loop
NUM_EPOCHS = 10

# Define number of folds for cross-validation
NUM_FOLDS = 5

# Momentum -> hyperparameter of Stochastic Gradient Descent (SGD) optimizer
MOMENTUM = 0.9


In [ ]:
# Load the data
df = pd.read_excel('/content/BSMDD_v3_textcleaned - 21K.xlsx')

df['label'] = df['label'].map({'Non Depressive': 0, 'Depressive': 1})
# Drop rows with any missing values
df = df.dropna(subset=['text', 'label'])
# drop rows that have identical values in all columns
df.drop_duplicates(inplace=True)

# Assuming your DataFrame is named 'df'
columns_to_drop = [col for col in df.columns if col not in ['text', 'label']]
df.drop(columns=columns_to_drop, inplace=True)

print(df.info())

label_counts = df['label'].value_counts()
print("Label Details:")
print(label_counts)

df.sample(n=10)  # Change the value of 'n' to the desired number of random samples

<class 'pandas.core.frame.DataFrame'>
Index: 21904 entries, 0 to 21909
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    21904 non-null  object 
 1   label   0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 513.4+ KB
None
Label Details:
Series([], Name: count, dtype: int64)


,text,label
17878,শ্রেণীতে ছিলাম দিয়েছিলাম হতাশাগ্রস্ত করেছিলাম...,NaN
1007,পুরানো আরবি প্রবাদ যেটি নিজেকে হত্যা নিজেকে সম...,NaN
1543,থ্রোওয়ে জীবন ভাল সত্যিই শেষ বার্লি বন্ধু ব্যক...,NaN
4374,দশকের শেষ পুরুষ নিখুঁত সম্পর্কে দিই নিজেকে গর্...,NaN
12088,বুকে উঠার সত্যিই দরকার দুঃখিত হায় বয়সে আত্মহ...,NaN
14024,বলছি সাহায্য কারো ধারণা মানে কিশোর বন্ধু ফোন ম...,NaN
4153,কিনা জানি লোকেদের সাথে সমস্যা বিরক্ত পূর্ব জার...,NaN
7829,বাবামা বিরতির সময় ইলেক্ট্রনিক সীমিত বাবামা স্...,NaN
2251,হাই বন্ধুর সম্পর্কে কথা বলিনি ঈশ্বর স্লার বন্ধ...,NaN
18911,এভাবেই প্রথমে এটাকে ঠাট্টা কখনোই ভাবলাম একটা স...,NaN


In [ ]:
df.head()

,text,label
0,মানসিক শারীরিকভাবে অসুস্থ ক্লান্ত পুরো জীবন শা...,NaN
1,দয়া সাথে থাকুন অত্যন্ত দীর্ঘ আপনাকে পড়তে উত্...,NaN
2,জানতাম সাথে ভুল লোক খারাপ জীবন কাটিয়েছে সম্পূ...,NaN
3,অনেটিভ ইংরেজি স্পিকারের অনুসরণ বিরক্তিকর অপ্রত...,NaN
4,অনেটিভ ইংরেজি স্পিকারের অনুসরণ বিরক্তিকর অপ্রত...,NaN


In [ ]:
# Sample 14,000 instances for each label
df_label_0 = df[df['label'] == 0].sample(n=14000)
df_label_1 = df[df['label'] == 1].sample(n=14000)

# Concatenate the sampled dataframes
df = pd.concat([df_label_0, df_label_1], ignore_index=True)

print(df.info())

label_counts = df['label'].value_counts()
print("Label Details:")
print(label_counts)

df.sample(n=10)  # Change the value of 'n' to the desired number of random samples

ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
X = df['text']
y = df['label']
print(X.info())
print(y.info())

<class 'pandas.core.series.Series'>
RangeIndex: 28000 entries, 0 to 27999
Series name: text
Non-Null Count  Dtype 
--------------  ----- 
28000 non-null  object
dtypes: object(1)
memory usage: 218.9+ KB
None
<class 'pandas.core.series.Series'>
RangeIndex: 28000 entries, 0 to 27999
Series name: label
Non-Null Count  Dtype
--------------  -----
28000 non-null  int64
dtypes: int64(1)
memory usage: 218.9 KB
None


In [ ]:
torch.cuda.empty_cache()

# Define the BERT model architecture

In [ ]:
# Define the BERT model architecture
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(DROPOUT)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return torch.sigmoid(logits)


# Set device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the BERT tokenizer and encode the text data
tokenizer = BertTokenizer.from_pretrained('sagorsarker/bangla-bert-base')
X_encodings = tokenizer(list(X), truncation=True, padding=True, max_length=512)

# Convert labels to tensor
y_tensor = torch.tensor(y.values)

# Initialize the StratifiedKFold
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_encodings['input_ids'], y_tensor), 1):
    print(f"Processing Fold: {fold}")

    # Convert X_encodings to a list of tuples
    X_encodings_list = list(X_encodings.items())

    # Split data into train and validation sets
    X_train_fold = {key: [value[i] for i in train_index] for key, value in X_encodings_list}
    X_val_fold = {key: [value[i] for i in val_index] for key, value in X_encodings_list}
    y_train_fold = y_tensor[train_index]
    y_val_fold = y_tensor[val_index]


    # Create PyTorch datasets
    train_dataset = TensorDataset(torch.tensor(X_train_fold['input_ids']),
                                  torch.tensor(X_train_fold['attention_mask']),
                                  torch.tensor(y_train_fold))
    val_dataset = TensorDataset(torch.tensor(X_val_fold['input_ids']),
                                torch.tensor(X_val_fold['attention_mask']),
                                torch.tensor(y_val_fold))

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


Processing Fold: 1


<ipython-input-12-946cccec6c68>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_train_fold))
<ipython-input-12-946cccec6c68>:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_val_fold))


Processing Fold: 2
Processing Fold: 3
Processing Fold: 4
Processing Fold: 5


In [ ]:
torch.cuda.empty_cache()


In [ ]:
# Define the BERT classifier model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert_model = bert_model
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert_model(input_ids, attention_mask)[1]  # Use the [CLS] token representation
        logits = self.fc(output)
        return logits

# Set the device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the BERT model
bert_model = BertModel.from_pretrained('sagorsarker/bangla-bert-base')


hidden_dim = bert_model.config.hidden_size
num_classes = 2

model = BERTClassifier(bert_model, hidden_dim, num_classes).to(DEVICE)


# Define the optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
criterion = nn.CrossEntropyLoss()

best_accuracy = 0.0

# Freeze BERT layer
for param in model.bert_model.parameters():
    param.requires_grad = False

# Unfreeze output layer
for param in model.fc.parameters():
    param.requires_grad = True

# Define lists to store metrics for each fold
fold_train_losses = []
fold_train_accuracies = []
fold_val_losses = []
fold_val_accuracies = []
fold_val_predictions = []
fold_val_labels = []

# Define lists to store train and validation metrics per epoch
train_losses_per_epoch = []
train_accuracies_per_epoch = []
val_losses_per_epoch = []
val_accuracies_per_epoch = []

for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch in train_loader:
      input_ids, attention_mask, labels = batch
      input_ids = input_ids.to(DEVICE)
      attention_mask = attention_mask.to(DEVICE)
      labels = labels.to(DEVICE)

      optimizer.zero_grad()

      logits = model(input_ids, attention_mask)
      _, predicted = torch.max(logits, 1)

      loss = criterion(logits, labels)
      loss.backward()
      optimizer.step()

      train_loss += loss.item() * input_ids.size(0)
      train_total += labels.size(0)
      train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for batch in val_loader:
          input_ids, attention_mask, labels = batch
          input_ids = input_ids.to(DEVICE)
          attention_mask = attention_mask.to(DEVICE)
          labels = labels.to(DEVICE)

          logits = model(input_ids, attention_mask)
          _, predicted = torch.max(logits, 1)

          loss = criterion(logits, labels)

          val_loss += loss.item() * input_ids.size(0)
          val_total += labels.size(0)
          val_correct += (predicted == labels).sum().item()

          fold_val_predictions.extend(predicted.cpu().numpy())
          fold_val_labels.extend(labels.cpu().numpy())

    val_accuracy = 100 * val_correct / val_total

    fold_train_losses.append(train_loss / train_total)
    fold_train_accuracies.append(train_accuracy)
    fold_val_losses.append(val_loss / val_total)
    fold_val_accuracies.append(val_accuracy)

    train_losses_per_epoch.append(train_loss / train_total)
    train_accuracies_per_epoch.append(train_accuracy)
    val_losses_per_epoch.append(val_loss / val_total)
    val_accuracies_per_epoch.append(val_accuracy)

    print('Epoch: {}/{} | Train Loss: {:.4f} | Train Accuracy: {:.2f}% | Val Loss: {:.4f} | Val Accuracy: {:.2f}%'.format(
        epoch + 1, NUM_EPOCHS, train_loss / train_total, train_accuracy, val_loss / val_total, val_accuracy))

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model.pth')

# Calculate performance metrics
average_accuracy = accuracy_score(fold_val_labels, fold_val_predictions)
average_precision = precision_score(fold_val_labels, fold_val_predictions)
average_recall = recall_score(fold_val_labels, fold_val_predictions)
average_f1_score = f1_score(fold_val_labels, fold_val_predictions)

# Print average metrics across folds
print('Average Metrics Across Folds:')
print('Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f} | F1 Score: {:.4f}'.format(
    average_accuracy, average_precision, average_recall, average_f1_score))

# Plotting the train and validation accuracy and loss graphs
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses_per_epoch, label='Train Loss')
plt.plot(val_losses_per_epoch, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies_per_epoch, label='Train Accuracy')
plt.plot(val_accuracies_per_epoch, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Some weights of the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 1/10 | Train Loss: 0.4214 | Train Accuracy: 80.44% | Val Loss: 0.3309 | Val Accuracy: 85.32%
Epoch: 2/10 | Train Loss: 0.3892 | Train Accuracy: 82.84% | Val Loss: 0.3057 | Val Accuracy: 86.93%
Epoch: 3/10 | Train Loss: 0.3773 | Train Accuracy: 83.46% | Val Loss: 0.3114 | Val Accuracy: 86.71%
Epoch: 4/10 | Train Loss: 0.3874 | Train Accuracy: 83.17% | Val Loss: 0.3881 | Val Accuracy: 83.55%
Epoch: 5/10 | Train Loss: 0.3754 | Train Accuracy: 83.86% | Val Loss: 0.4295 | Val Accuracy: 81.16%
Epoch: 6/10 | Train Loss: 0.3698 | Train Accuracy: 83.76% | Val Loss: 0.3062 | Val Accuracy: 86.86%
Epoch: 7/10 | Train Loss: 0.3839 | Train Accuracy: 83.18% | Val Loss: 0.3037 | Val Accuracy: 87.32%


In [ ]:
# Print average metrics across folds
print('Average Metrics Across Folds:')
print('Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f} | F1 Score: {:.4f}'.format(
    average_accuracy, average_precision, average_recall, average_f1_score))

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data
df = pd.read_excel('0. All Datasets.xlsx')
df['label'] = df['label'].map({'Non Depressive': 0, 'Depressive': 1})
df = df.dropna(subset=['text', 'label'])
df.drop_duplicates(inplace=True)
columns_to_drop = [col for col in df.columns if col not in ['text', 'label']]
df.drop(columns=columns_to_drop, inplace=True)

label_counts = df['label'].value_counts()
print("Label Details:")
print(label_counts)

# Sample 14,000 instances for each label
df_label_0 = df[df['label'] == 0].sample(n=14000, random_state=42)
df_label_1 = df[df['label'] == 1].sample(n=14000, random_state=42)

# Concatenate the sampled dataframes
df = pd.concat([df_label_0, df_label_1], ignore_index=True)

X = df['text']
y = df['label']

In [ ]:
# Define the BERT classifier model
class BERTClassifierWrapper(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifierWrapper, self).__init__()
        self.bert_model = bert_model
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert_model(input_ids, attention_mask)[1]  # Use the [CLS] token representation
        logits = self.fc(output)
        return logits

    def fit(self, input_ids, attention_mask, labels, criterion, optimizer, num_epochs, batch_size):
        dataset = TensorDataset(input_ids, attention_mask, labels)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        self.train()
        for epoch in range(num_epochs):
            running_loss = 0.0
            for i, batch in enumerate(data_loader):
                input_ids_batch, attention_mask_batch, labels_batch = batch
                input_ids_batch = input_ids_batch.to(DEVICE)
                attention_mask_batch = attention_mask_batch.to(DEVICE)
                labels_batch = labels_batch.to(DEVICE)

                optimizer.zero_grad()

                outputs = self(input_ids_batch, attention_mask_batch)
                loss = criterion(outputs, labels_batch)

                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            print(f"Epoch {epoch+1}/{num_epochs} Loss: {running_loss / len(data_loader)}")

    def predict(self, input_ids, attention_mask):
        dataset = TensorDataset(input_ids, attention_mask)
        data_loader = DataLoader(dataset, batch_size=32, shuffle=False)

        self.eval()
        predictions = []
        with torch.no_grad():
            for batch in data_loader:
                input_ids_batch, attention_mask_batch = batch
                input_ids_batch = input_ids_batch.to(DEVICE)
                attention_mask_batch = attention_mask_batch.to(DEVICE)

                outputs = self(input_ids_batch, attention_mask_batch)
                _, predicted_labels = torch.max(outputs, dim=1)

                predictions.extend(predicted_labels.cpu().numpy())

        return predictions

# Load the BERT tokenizer and encode the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
X_encodings = tokenizer(list(X), truncation=True, padding=True, max_length=512)

# Convert labels to tensor
y_tensor = torch.tensor(y.values)

# Set device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define hyperparameters and their possible values for grid search
parameters = {
    'num_epochs': [5, 10],
    'batch_size': [32, 64],
    'learning_rate': [0.001, 0.01],
    'momentum': [0.9, 0.95],
    'dropout': [0.1, 0.2],
}

# Initialize the BERT model
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')
hidden_dim = bert_model.config.hidden_size
num_classes = 2

# Create the BERT classifier model
model = BERTClassifierWrapper(bert_model, hidden_dim, num_classes).to(DEVICE)

# Define the optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Define the grid search
grid_search = GridSearchCV(model, parameters, cv=5, scoring='accuracy')

# Run grid search
grid_search.fit(X_encodings['input_ids'], X_encodings['attention_mask'], y_tensor, criterion, optimizer)

# Print the best parameters and score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)